In [1]:
from poisson_fem.mesh import PoissonFEM
import numpy as np
import scipy.sparse as sps
import scipy.sparse.linalg as lg
import time
import petsc4py
import sys
petsc4py.init(sys.argv)
from petsc4py import PETSc

In [2]:
mesh = PoissonFEM.RectangularMesh(np.ones(16)/16)
# mesh.plot()

In [3]:
def origin(x):
    return np.abs(x[0]) < np.finfo(float).eps and np.abs(x[1]) < np.finfo(float).eps
mesh.setEssentialBoundary(origin)

In [4]:
# Precomputations
Ke = mesh.compGlobStiffStencil()
K0 = PETSc.Mat().createAIJ(size=(mesh.nEq, mesh.nEq), nnz=2394)
lmbda = PETSc.Vec().createSeq(mesh.nCells)
Kvec = PETSc.Vec().createSeq(mesh.nEq**2)
x = PETSc.Vec().createSeq(mesh.nEq)
rhs = PETSc.Vec().createSeq(mesh.nEq)

# Get sparsity pattern of stiffness matrix vector
lmbda.setValues(range(mesh.nCells), np.ones(mesh.nCells))
Ke.mult(lmbda, Kvec)
nonzero_K = np.nonzero(Kvec.array)
Ktmp = sps.csr_matrix(np.reshape(Kvec.array, (mesh.nEq, mesh.nEq), order='F'))
indptr = Ktmp.indptr.copy()
indices = Ktmp.indices.copy()

# Set up solver
ksp = PETSc.KSP().create()
ksp.setType('preonly')
pc = ksp.getPC()
pc.setType('cholesky')
ksp.setFromOptions() #???

# for scipy comparison
Kes = Ke.getValuesCSR()
Kes = sps.csr_matrix((Kes[2], Kes[1], Kes[0]))

In [5]:
N = 1e3

In [6]:
start = time.time()
for n in range(int(N)):
    # Stiffness matrix assembly, lmbda = [1, 1, ..., 1]
    lmbda.setValues(range(mesh.nCells), np.ones(mesh.nCells))
    rhs.setValues(range(mesh.nEq), np.ones(mesh.nEq))
    Ke.mult(lmbda, Kvec)
#     K = sps.csr_matrix(np.reshape(Kvec.array, (mesh.nEq, mesh.nEq), order='F'))
    K0.setValuesCSR(indptr, indices, Kvec.getValues(nonzero_K))
    K0.assemblyBegin()
    K0.assemblyEnd()

    # solving
    ksp.setOperators(K0)
    ksp.solve(rhs, x)
petsc_time = (time.time() - start)/N
print('PETSc time = ', petsc_time)

PETSc time =  0.0002717530727386475


In [7]:
start = time.time()
for n in range(int(N)):
    Kvecs = Kes @ np.ones(mesh.nCells)
    K1 = sps.csr_matrix(np.reshape(Kvec.array, (mesh.nEq, mesh.nEq), order='F'))
    x1 = lg.spsolve(K1, np.ones(mesh.nEq))
scipy_time = (time.time() - start)/N
print('scipy time = ', scipy_time)

scipy time =  0.0017897694110870362


In [8]:
diff = np.linalg.norm(x.array - x1)/np.linalg.norm(x1)
print('result difference = ', diff)
print('PETSc speedup = ', scipy_time/petsc_time)

result difference =  5.895038462914976e-14
PETSc speedup =  6.586013519737852
